# Notebook 2: Indexación en ChromaDB

Este notebook cubre la indexación del dataset en ChromaDB:
- Cargar dataset preparado
- Generar embeddings semánticos
- Indexar en ChromaDB
- Verificar la indexación

**Flujo**: Dataset → Embeddings → ChromaDB

## 1. Importar Librerías

In [41]:
import json
from pathlib import Path
import chromadb
from chromadb.utils import embedding_functions
import pandas as pd

## 2. Configuración

In [42]:
BASE_DIR = Path.cwd()
DATASET_FILE = BASE_DIR / "partidas_embedding_ready.json"
CHROMA_DIR = BASE_DIR / "chroma_propaher_db"

COLLECTION_NAME = "partidas_propaher"
EMBEDDING_MODEL = "paraphrase-multilingual-MiniLM-L12-v2"

print(f"Directorio ChromaDB: {CHROMA_DIR}")
print(f"Modelo de embeddings: {EMBEDDING_MODEL}")

Directorio ChromaDB: /Users/alexmartin/Documents/KC_projects/propaher/chroma_propaher_db
Modelo de embeddings: paraphrase-multilingual-MiniLM-L12-v2


## 3. Cargar Dataset

In [43]:
with open(DATASET_FILE, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

partidas = dataset["partidas"]

print(f"Total partidas: {len(partidas)}")
print(f"Metadata: {dataset.get('metadata', {})}")

if partidas:
    print("\nEjemplo de partida:")
    ejemplo = partidas[0]
    for key, value in ejemplo.items():
        print(f"  {key}: {value}")

Total partidas: 412
Metadata: {'version': '1.0', 'fecha_creacion': '2026-02-08T19:19:03.421855', 'total_partidas': 412, 'fuente': '25  MEMORA Guixeres Badalona (Tanatorio) Modif. 2.xlsx'}

Ejemplo de partida:
  capitulo: ESCOMESA GENERAL
  concepto_base: Vàlvula de comporta de claveguera estacionari, per a muntatge en brides
  concepto_normalizado: valvula comporta claveguera estacionari muntatge brides
  descripcion_tecnica: de 65 mm de diàmetre, PN-16, amb joc d'accessoris. Totalment instal·lada. Marca/model: ARC o similar
  unidad: ud
  cantidad: 1.0
  precio_unitario: 185.89423076923075
  importe: 185.89423076923075
  origen: 25  MEMORA Guixeres Badalona (Tanatorio) Modif. 2
  archivo: 25  MEMORA Guixeres Badalona (Tanatorio) Modif. 2.xlsx


## 4. Inicializar ChromaDB

In [44]:
client = chromadb.PersistentClient(path=str(CHROMA_DIR))

collections = client.list_collections()
print(f"Colecciones existentes: {len(collections)}")
for col in collections:
    print(f"  {col.name} ({col.count()} documentos)")

Colecciones existentes: 1
  partidas_propaher (24 documentos)


## 5. Crear/Recrear Colección

In [45]:
try:
    client.delete_collection(COLLECTION_NAME)
    print(f"Colección '{COLLECTION_NAME}' eliminada")
except:
    print(f"Colección '{COLLECTION_NAME}' no existía")

embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_MODEL
)

collection = client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_fn,
    metadata={"hnsw:space": "cosine"}
)

print(f"Colección '{COLLECTION_NAME}' creada")
print(f"Métrica de similitud: Cosine")

Colección 'partidas_propaher' eliminada
Colección 'partidas_propaher' creada
Métrica de similitud: Cosine


## 6. Preparar Datos para Indexación

In [46]:
documents = []
metadatas = []
ids = []

for i, p in enumerate(partidas):
    doc = f"{p.get('concepto_base', '')} | {p.get('concepto_normalizado', '')}"
    documents.append(doc)
    
    metadatas.append({
        "capitulo": str(p.get("capitulo", "")),
        "concepto_base": str(p.get("concepto_base", "")),
        "unidad": str(p.get("unidad", "")),
        "precio_unitario": float(p.get("precio_unitario", 0)),
        "origen": str(p.get("origen", "")),
        "archivo": str(p.get("archivo", ""))
    })
    
    ids.append(f"partida_{i}")

print(f"Datos preparados:")
print(f"  Documentos: {len(documents)}")
print(f"  Metadata: {len(metadatas)}")
print(f"  IDs: {len(ids)}")

Datos preparados:
  Documentos: 412
  Metadata: 412
  IDs: 412


## 7. Indexar en ChromaDB

In [47]:
batch_size = 100
total_batches = (len(documents) + batch_size - 1) // batch_size

print(f"Iniciando indexación...")
print(f"  Total documentos: {len(documents)}")
print(f"  Tamaño de lote: {batch_size}")
print(f"  Total lotes: {total_batches}")

for i in range(0, len(documents), batch_size):
    batch_num = (i // batch_size) + 1
    
    batch_docs = documents[i:i+batch_size]
    batch_metas = metadatas[i:i+batch_size]
    batch_ids = ids[i:i+batch_size]
    
    collection.add(
        documents=batch_docs,
        metadatas=batch_metas,
        ids=batch_ids
    )
    
    print(f"  Lote {batch_num}/{total_batches} indexado ({len(batch_docs)} documentos)")

print(f"Indexación completada: {collection.count()} documentos")

Iniciando indexación...
  Total documentos: 412
  Tamaño de lote: 100
  Total lotes: 5
  Lote 1/5 indexado (100 documentos)
  Lote 2/5 indexado (100 documentos)
  Lote 3/5 indexado (100 documentos)
  Lote 4/5 indexado (100 documentos)
  Lote 5/5 indexado (12 documentos)
Indexación completada: 412 documentos


## 8. Verificar Indexación

In [48]:
collection_info = client.get_collection(COLLECTION_NAME)

print(f"Información de la colección:")
print(f"  Nombre: {collection_info.name}")
print(f"  Total documentos: {collection_info.count()}")

sample = collection_info.get(
    ids=["partida_0", "partida_1", "partida_2"],
    include=["documents", "metadatas"]
)

print("\nDocumentos de ejemplo indexados:")
for i, (doc, meta) in enumerate(zip(sample["documents"], sample["metadatas"])):
    print(f"\n[{i+1}] {meta['concepto_base'][:60]}")
    print(f"    Precio: {meta['precio_unitario']}€/{meta['unidad']}")
    print(f"    Origen: {meta['origen']}")

Información de la colección:
  Nombre: partidas_propaher
  Total documentos: 412

Documentos de ejemplo indexados:

[1] Vàlvula de comporta de claveguera estacionari, per a muntatg
    Precio: 185.89423076923077€/ud
    Origen: 25  MEMORA Guixeres Badalona (Tanatorio) Modif. 2

[2] Vàlvula de comporta de claveguera estacionari, per a muntatg
    Precio: 152.68725490196076€/ud
    Origen: 25  MEMORA Guixeres Badalona (Tanatorio) Modif. 2

[3] Vàlvula de retenció de disc, per a instal·lacions d'escomesa
    Precio: 100.50840909090908€/ud
    Origen: 25  MEMORA Guixeres Badalona (Tanatorio) Modif. 2


## 9. Prueba de Búsqueda Rápida

In [49]:
query = "tubo corrugado"

print(f"Búsqueda de prueba: '{query}'\n")

resultados = collection.query(
    query_texts=[query],
    n_results=3
)

print("Resultados:")
for i, (meta, dist) in enumerate(zip(
    resultados["metadatas"][0],
    resultados["distances"][0]
)):
    similitud = 1 - dist
    print(f"\n{i+1}. [{similitud:.1%}] {meta['concepto_base']}")
    print(f"   {meta['precio_unitario']}€/{meta['unidad']}")
    print(f"   {meta['origen']}")

Búsqueda de prueba: 'tubo corrugado'

Resultados:

1. [65.2%] Tub de polietilè flexible corrugat amb interior llis per a distribució subterrània,
   6.5296€/m
   25  MEMORA Guixeres Badalona (Tanatorio) Modif. 2

2. [65.2%] Tub de polietilè flexible corrugat amb interior llis per a distribució subterrània,
   3.1040624999999995€/m
   25  MEMORA Guixeres Badalona (Tanatorio) Modif. 2

3. [65.2%] Tub de polietilè flexible corrugat amb interior llis per a distribució subterrània,
   4.093833333333333€/m
   25  MEMORA Guixeres Badalona (Tanatorio) Modif. 2


## 10. Estadísticas de la Indexación

In [50]:
all_data = collection.get(include=["metadatas"])
df_indexed = pd.DataFrame(all_data["metadatas"])

print(f"Total documentos indexados: {len(df_indexed)}")

print("\nEstadísticas de precios:")
display(df_indexed['precio_unitario'].describe())

print("\nDistribución por unidad:")
display(df_indexed['unidad'].value_counts())

print("\nDistribución por origen:")
display(df_indexed['origen'].value_counts())

Total documentos indexados: 412

Estadísticas de precios:


count      412.000000
mean      1057.624836
std       4778.921769
min          1.698446
25%         29.215754
50%         76.923077
75%        347.638306
max      55189.873471
Name: precio_unitario, dtype: float64


Distribución por unidad:


unidad
ud    299
m     113
Name: count, dtype: int64


Distribución por origen:


origen
25  MEMORA Guixeres Badalona (Tanatorio) Modif. 2    412
Name: count, dtype: int64